In [1]:
# change workdir to be root directory of covariance_resolution software
workdir = '/home/patrick/Master_Lenovo/Academic/ORNL Research Position (August 2016 - June 2017)/linear_Gaussian_error_propagation/covariance_resolution_method'
%cd {workdir}

/home/patrick/Master_Lenovo/Academic/ORNL Research Position (August 2016 - June 2017)/linear_Gaussian_error_propagation/covariance_resolution_method


In [2]:
# Import necessary python modules
import numpy as np
from matplotlib import pyplot as plt

import compute_covariance_matrix as cov_mat
import ARCS_error_propagation as arcs
import compute_vQ_to_HKL_basis_change_matrix as HKL_basis
import plot_covariance_ellipse as plot_cov
import avg_cov_mat_across_angle_sweep as avg_cov


# define physical constants:

# m = mass of neutron (kg)
m = 1.674929*(10**-27)

# hbar = reduced Planck's constant (J s)
hbar = 1.0545718*(10**-34)

First, we need to define the required instrumental parameters for our experiment:

In [3]:
# set instrument (currently, only arcs supported)
instrument = "arcs"

# a bit of hacking:
# define distances (eventually, this will be done automatically)
L12 = 18.50 - 11.83  # distance from beam monitor 1 to 2
Lms = 13.60  # distance from moderator to sample
Lsp = 3.45  # distance from sample to detector pixel -- this will also need to be changed eventually to take a particular values


# incident beam energy
Ei_meV = 150.0  # meV

# set u, v sample orientation vectors
u = [1, 0, 2]
v = [1, 0, 0]

# define uncertainties
sigma_t12 = 10.0*10**-6
sigma_theta = 1.5 / 360.0 * 2.0*np.pi
sigma_phi = 1.5 / 360.0 * 2.0*np.pi

var_t12 = sigma_t12**2
var_theta = sigma_theta**2
var_phi = sigma_phi**2


Next, we need to define our sample properties

In [4]:
# lattice parameters
a = 5.0
b = 5.0
c = 5.0

# lattice vectors
a1 = np.array([a, 0, 0])
a2 = np.array([0, b, 0])
a3 = np.array([0, 0, c])

# create a list of lattice vectors
lattice = [a1, a2, a3]

# sample rotation angle
angle = 0.0

Now, we will take a specified point in HKLE space as our input, then compute the corresponding instrument coordinates

In [21]:
# specify HKLE point:
HK = 1.5
H = np.sqrt(HK**2 / 2.0)
K = H
L = 0.0
E = 25.0 # meV

# now, convert to real units (inverse angstroms)
h = H * 2*np.pi / a
k = K * 2*np.pi / b
l = L * 2*np.pi / c

# make array of hklE values
HKLE = np.array([h, k, l, E])

# compute instrument coordinates
instr_coords = avg_cov.HKLE_to_instrumentCoords(lattice, u, v, angle, HKLE, Ei_meV, L12, Lms, Lsp)

print (instr_coords)



[1.8925468811915387, 0.84106867056793044, 0.0012451112614357287, 0.0032442495164199591]


Extra Information:  This section will be refined later to be more user-friendly.  Right now, it is just an information dump of particular details for which point in instrument space / Q,E space / HKLE space is going to be analyzed

In [22]:
#tof = 3900.9  # time-of-flight (milliseconds)
#t = tof*10**-6  # tof (in seconds)
t = instr_coords[3]

#theta = 2.24506  # polar angle (relative to beam / z-direction -- radians)
#phi = -0.56543  # azimuthal angle (relative to x-direction -- radians)
theta = instr_coords[0]
phi = instr_coords[1]


# Now, compute some additional quantities
Ei = arcs.meV_to_joules(Ei_meV)  # Ei in Joules
vi = arcs.vi_from_Ei(Ei)  # incident neutron speed (m/s)
#t12 = arcs.t12_from_vi(vi)  # time for travel from 1st to 2nd beam monitor
t12 = instr_coords[2]
vf = arcs.vf_from_tof_and_t12(t, t12, Lsp, Lms, L12)
Ef = 0.5*m*vf**2

We now need to actually compute the covariance matrix

In [28]:
# get Jacobian matrix
#J = arcs.setup_jacobian(vi, vf, Ei, Ef, theta, phi, L12, Lms, Lsp, tof, t12)
J = arcs.setup_jacobian(vi, vf, Ei, Ef, theta, phi, L12, Lms, Lsp, t*1.08, t12)

# get instrumental parameters matrix
M = cov_mat.setup_params_matrix(var_t12, var_theta, var_phi)

# get vQE covariance matrix
Sigma_vQE = cov_mat.get_sigma_vQE(J, M)

# get HKLE covariance matrix
Sigma_HKLE = cov_mat.get_sigma_HKLE(Sigma_vQE, lattice, u, v, angle)

# get inverse HKLE covariance matrix (for plotting)
SigmaInv_HKLE = cov_mat.get_sigma_inv_HKLE(Sigma_HKLE)



E_partial_t12 = -1.00472111013e-16



In [29]:
print("Sigma_vQE = \n")
print(Sigma_vQE)

print("\nSigma_HKLE = \n")
print(Sigma_HKLE)

Sigma_vQE = 

[[  2.82675403e-02  -9.99938102e-03   1.05754179e-02  -4.75848145e-01]
 [ -9.99938102e-03   2.60316107e-02   1.18236767e-02  -5.32014400e-01]
 [  1.05754179e-02   1.18236767e-02   3.81351360e-02  -1.90623512e-01]
 [ -4.75848145e-01  -5.32014400e-01  -1.90623512e-01   3.93313255e+01]]

Sigma_HKLE = 

[[  2.17807251e-02  -1.42314272e-02   2.39821248e-03  -2.70851548e-01]
 [ -1.42314272e-02   2.60316107e-02   6.10355879e-03  -4.23363608e-01]
 [  2.39821248e-03   6.10355879e-03   4.46219512e-02  -3.05024119e-01]
 [ -4.27711622e-01  -6.68549012e-01  -4.81674782e-01   3.93313255e+01]]


In [30]:
# select two variables (from H,K,L,E) to plot: (0,1,2,3 = H,K,L,E)
x1 = 2
x1_title = ""
x2 = 3
x2_title = ""

if x1 == 0:
    x1_title = "H-direction (A^-1)"
elif x1 == 1:
    x1_title = "K-direction (A^-1)"
elif x1 == 2:
    x1_title = "L-direction (A^-1)"
elif x1 == 3:
    x1_title = "E (meV)"

if x2 == 0:
    x2_title = "H-direction (A^-1)"
elif x2 == 1:
    x2_title = "K-direction (A^-1)"
elif x2 == 2:
    x2_title = "L-direction (A^-1)"
elif x2 == 3:
    x2_title = "E (meV)"
    
# extract submatrix:
A = plot_cov.get_2D_subcovariance(Sigma_vQE, x1, x2)

# specify degrees of freedom and significance level
alpha = 0.5
k = 2
# compute critical chi-squared value
chi2 = plot_cov.get_critical_chi_squared(k, alpha)

# plot covariance matrix:
plot_cov.plot_quadform(A, x1, x2, chi2, x1_title, x2_title, 0)
#plot_cov.plot_quadform_method2(A, x1, x2, x1_title, x2_title)